In [1]:
import numpy as np
import pandas as pd
import math
import os
dpath = os.path.join( os.curdir, 'data')
path = os.curdir

# Расчет несущей способности буровой сваи

# Исходные данные

**Грунты:**

In [2]:
dat = 'Грунт2.csv'
NL = -0.52 # отметка поверхности природного рельефа
DL = 0 # отметка планировки
soil = pd.read_table(os.path.join(path, dat), sep=';')

**Параметры сваи:**

In [3]:
d = 880 # диаметр сваи, мм
Yc = 1 # Коэффициент условий работы сваи

# Расчёт

In [4]:
def Rf(tbl, il, z):
    tbl.index = pd.to_numeric(tbl.index)
    new_index = np.unique(list(tbl.index) + [z])
    new_tbl = tbl.reindex(new_index).interpolate('index').loc[z]
    new_tbl.index = pd.to_numeric(new_tbl.index)
    new_id = np.unique(list(pd.to_numeric(new_tbl.index)) + [il])
    rf = new_tbl.reindex(new_id).interpolate('index').loc[il]
    return rf
def fi(t, il, z):
    if t == 'песчаный':
        tbl = pd.read_table(os.path.join(dpath, 'Песчаные_7.3.csv'), sep=';', index_col=0)
    else:
        tbl = pd.read_table(os.path.join(dpath, 'Глинистые_7.3.csv'), sep=';', index_col=0)
    f = Rf(tbl, il, z)
    return f

def ri(t, il, z):
    if t == 'Глинистые_7.8':
        tbl = pd.read_table(os.path.join(dpath, t + '.csv'), sep=';', index_col=0).interpolate()
        r = Rf(tbl, il, z)
        print('R=', round(r, 3), 'кПа')
    else:
        tbl_1_2 = pd.read_table(os.path.join(dpath, t + '_1_2' + '.csv'), sep=';').transpose()
        tbl_1_2.index = pd.to_numeric(tbl_1_2.index)
        new_index = np.unique(list(tbl_1_2.index) + [il])
        alpha_1_2 = tbl_1_2.reindex(new_index).interpolate('index').loc[il]
        print('alpha_1=', round(alpha_1_2[0], 3))
        print('alpha_2=', round(alpha_1_2[1], 3))
        tbl_3 = pd.read_table(os.path.join(dpath, t + '_3' + '.csv'), sep=';', index_col=0)
        alpha_3 = Rf(tbl_3, il, z / d * 1000)
        print('alpha_3=', round(alpha_3, 3))
        tbl_4 = pd.read_table(os.path.join(dpath, t + '_4' + '.csv'), sep=';', index_col=0)
        alpha_4 = Rf(tbl_4, il, d / 1000)
        print('alpha_4=', round(alpha_4, 3))
        Y1_ = soil['Y'][n]
        print('Y1_=', round(Y1_, 3), 'кН/м^3')
        Y1 = sum(soil['Y'].head(-1) * soil['h'].head(-1)) / sum(soil['h'].head(-1))
        print('Y1=', round(Y1, 3), 'кН/м^3')
        print('h=', round(z, 3), 'м')
        r = 0.75 * alpha_4 * (alpha_1_2[0] * Y1_ * d / 1000 + alpha_1_2[1] * alpha_3 * Y1 * z)
    return r

In [5]:
if DL > NL:
    cut = 'подсыпка'
elif DL < NL:
    cut = 'срезка'
else:
    cut = 'нет'
cutv = DL - NL

if abs(cutv) < 3.0:
    CL = NL
elif cutv < 0:
    CL = DL + 3.0
else:
    CL = DL - 3.0
soil['h'] = np.append(soil['Верх'].head(-1).to_numpy() - soil['Верх'].tail(-1).to_numpy(), 0)
soil['z'] = abs(soil['Верх'].to_numpy() - soil['h'].to_numpy()/2 - CL)

In [6]:
n = len(soil)-1
A = math.pi * (d/1000) ** 2 / 4
u = math.pi * (d/1000)
Lp = soil['Верх'][0] - soil['Верх'][n]
pr = soil['IL/fi'][n]
if soil['Грунт'][n] == 'песчаный':
    if cut == 'срезка':
        zr = DL - soil['Верх'][n]
    else:
        zr = NL - soil['Верх'][n]
    tab = 'Песчаные_7.7'
    R = ri(tab, soil['IL/fi'][n], zr)
    tblmax = pd.read_table(os.path.join(dpath, 'Песчаные_7.2.csv'), sep=';', index_col=0)
    Rmax = Rf(tblmax, soil['IL/fi'][n-1], soil['z'][n]) * soil['k'][n]
    if Rmax > 20000:
        Rmax = 20000
    if R > Rmax:
        R = Rmax
    print('R_Tab_7.2=', round(Rmax, 3), 'кПа')
else:
    zr = soil['z'][0] - soil['h'][0] / 2 + Lp
    tab = 'Глинистые_7.8'
    R = ri(tab, soil['IL/fi'][n], zr)
f = np.zeros(n)
print('R_Tab_'+ tab, '=', round(R, 3), 'кПа')
for i in range(0, n):
    f[i] = fi(soil['Грунт'][i], soil['IL/fi'][i], soil['z'][i])
soil['f/R'] = np.append(f, R)
skin = u * sum(soil['YR'] * soil['f/R'] * soil['h'] * soil['k'])
base = soil['YR'][n] * R * A
Fd = Yc * (base + skin)

alpha_1= 55.41
alpha_2= 99.42
alpha_3= 0.679
alpha_4= 0.246
Y1_= 10.5 кН/м^3
Y1= 13.374 кН/м^3
h= 22.83 м
R_Tab_7.2= 11047.52 кПа
R_Tab_Песчаные_7.7 = 3893.203 кПа


# Результаты

In [7]:
print('Отметка поверхности природного рельефа NL=', NL)
print('Планировка:', cut, np.round(cutv, 2), 'м')
print('Условная отметка:', CL, 'м')
print('Коэффициент условий работы сваи Yc=', Yc)
print('Параметры слоёв грунта:')
print(soil)
print('Первая строка в таблице - для низа ростверка')
print('Последняя строка в таблице - для низа сваи')
print('Обозначения по п.7.2.10, 7.2.11 СП 24.13330.2021 Свайные фундаменты')
print('k - коэффициенты к таблицам 7.2, 7.3,  СП 24.13330.2021 Свайные фундаменты')
print('fi - для низа сваи при песчаных грунтах, градусы')
print('R - расчетное сопротивление грунта под нижним концом сваи, кПа')
print('Параметры сваи:')
print('Площадь опирания A=', round(A, 3), 'м^2')
print('Периметр сечения ствола u=', round(u, 3), 'м')
print('Длина сваи:', Lp, 'м')
print('глубина заложения нижнего конца сваи', round(zr, 2), 'м')
print('Коэффициент надежности yc=', round(Yc, 3))
print('Результаты:')
print('Боковая поверхность несёт:', round(skin, 2), 'кН')
print('Конец сваи несёт:', round(base, 2), 'кН')
print('Несущая способность сваи Fd=:', round(Fd, 2), 'кН')

Отметка поверхности природного рельефа NL= -0.52
Планировка: подсыпка 0.52 м
Условная отметка: -0.52 м
Коэффициент условий работы сваи Yc= 1
Параметры слоёв грунта:
   ИГЭ   Верх      Грунт  IL/fi      Y    k   YR     h       z          f/R
0   2'  -6.35  глинистый   0.66  18.64  1.0  0.7  2.00   6.830    13.366000
1   2'  -8.35  глинистый   0.66  18.64  1.0  0.7  0.83   8.245    13.600000
2   5'  -9.18  глинистый   0.35  19.42  1.0  0.7  2.10   9.710    39.782500
3   3' -11.28  глинистый   0.36  20.01  1.0  0.7  1.20  11.360    39.996800
4   7' -12.48   песчаный   0.30  10.40  1.3  0.7  2.00  12.960    48.960000
5   7' -14.48   песчаный   0.30  10.40  1.3  0.7  0.50  14.210    50.210000
6   6' -14.98   песчаный   0.30   9.30  1.0  0.7  2.00  15.460    51.460000
7   6' -16.98   песчаный   0.30   9.30  1.0  0.7  1.40  17.160    53.160000
8   9' -18.38   песчаный   0.20  10.50  1.3  0.7  2.00  18.860    77.404000
9   9' -20.38   песчаный   0.20  10.50  1.3  0.7  2.00  20.860    80.204000